In [1]:
import numpy as np
from numpy import array
# import matplotlib as plt
from random import sample
from numpy import zeros
from math import sqrt
import _pickle as cPickle
import h5py

In [2]:
movies = 976
users = 6078
reviews = 5
active = 51

# Data Loading

In [3]:
dataset = cPickle.load( open( "dataset.pkl", "rb" ) )

In [4]:
def matr(shp):
    temp = zeros(shp)
    temp[:,0,:] = range(shp[2])
    temp[:,:,0] = range(shp[1])
    return temp

def matr1(shp):
    temp = zeros(shp)
    temp[0,:] = range(shp[1])
    temp[:,0] = range(shp[0])
    return temp

In [5]:
dataset.shape

(5, 485, 977)

# Separate Data in Active and Passive

In [7]:
def data_gen():
    lst = []
    x = list(range(1,485))
    for i in range(0,9*51,51):
        act_rand,pass_rand = array([0] + x[i:i+active]),array([0] + x[:i] + x[active+i:])
        active_user,passive_user = dataset[:,act_rand],dataset[:,pass_rand]
        cPickle.dump( active_user, open( "active_"+ str(i) +".pkl", "wb" ) )
        cPickle.dump( passive_user, open( "passive_"+ str(i)+".pkl", "wb" ) )
        lst.append(i)

    i += 51
    act_rand,pass_rand = array([0] + x[i:]),array([0] + x[:i])
    active_user,passive_user = dataset[:,act_rand],dataset[:,pass_rand]
    cPickle.dump( active_user, open( "active_"+ str(i) +".pkl", "wb" ) )
    cPickle.dump( passive_user, open( "passive_"+ str(i)+".pkl", "wb" ) )
    lst.append(i)
    return lst

In [6]:
dataset[-1,1:,0]

array([  1.00000000e+00,   3.00000000e+00,   7.00000000e+00,
         2.60000000e+01,   3.70000000e+01,   4.80000000e+01,
         6.90000000e+01,   7.00000000e+01,   9.20000000e+01,
         9.60000000e+01,   1.02000000e+02,   1.04000000e+02,
         1.37000000e+02,   1.46000000e+02,   1.65000000e+02,
         1.66000000e+02,   1.68000000e+02,   1.72000000e+02,
         1.76000000e+02,   2.06000000e+02,   2.32000000e+02,
         2.40000000e+02,   2.42000000e+02,   2.43000000e+02,
         2.45000000e+02,   2.87000000e+02,   3.01000000e+02,
         3.04000000e+02,   3.28000000e+02,   3.37000000e+02,
         3.51000000e+02,   3.57000000e+02,   3.73000000e+02,
         4.08000000e+02,   4.52000000e+02,   4.65000000e+02,
         4.77000000e+02,   4.85000000e+02,   4.99000000e+02,
         5.03000000e+02,   5.20000000e+02,   5.28000000e+02,
         5.29000000e+02,   5.32000000e+02,   5.51000000e+02,
         5.54000000e+02,   5.55000000e+02,   5.71000000e+02,
         5.92000000e+02,

In [8]:
list_file = data_gen()

In [11]:
cPickle.dump( list_file, open( "list_file.pkl", "wb" ) )

# Divide Active in Train and Test

In [34]:
for i in range(len(list_file)):
    active_user,pasive_user = cPickle.load( open( "active_"+ str(list_file[i]) +".pkl", "rb" ) ),cPickle.load( open( "passive_"+ str(list_file[i])+".pkl", "rb" ) )
    ratio_train = (70 * movies)/100
    ratio_test = movies-ratio_train
    y = list(range(1,977))
    np.random.shuffle(y)
    train_rand,test_rand = array([0] + y[:int(ratio_train)]),array([0] + y[int(ratio_train):])
    train,test = active_user[:,:,train_rand],active_user[:,:,test_rand]
    cPickle.dump( train, open( "train_"+ str(i) +".pkl", "wb" ) )
    cPickle.dump( test, open( "test_"+ str(i)+".pkl", "wb" ) )

# Similarity Finding

In [35]:
def simlarity(rows,passive_users,user_list,k):
    users = zeros((reviews,k))
    for row1,passive_user,user in zip(rows,passive_users,users):
        k_neighbr,idx,arr = np.zeros(passive_user.shape[0]),0,user_list
#         print(passive_user.shape)
        for row2 in passive_user[:,1:]:
            temp1,temp2,temp3 = 0,0,0
            mu1 = np.sum([row1], axis=1)[0]/row1.nonzero()[0].shape[0]
            mu2 = np.sum([row2], axis=1)[0]/row2.nonzero()[0].shape[0]
            for i in range(len(row1)):
                if row1[i] is not 0 and row2[i] is not 0:
                    temp1 += ((row1[i] - mu1) * (row2[i] - mu2))
                    temp2 += (row1[i] - mu1) * (row1[i] - mu1)
                    temp3 += (row2[i] - mu2) * (row2[i] - mu2)
            k_neighbr[idx] = temp1/(sqrt(temp2) * sqrt(temp3))
            idx+=1
        user[:] = arr[k_neighbr.argsort()][-k:][::-1]
    return np.unique(users.flatten())

def k_neighbor(data,passive_user,k=20):
#     users = matr1((data.shape[1],(reviews*k)+1))
    users = matr1((data.shape[1],60))
    for idx in range(1,data[0,1:,0].shape[0]+1):
        temp,users[idx,0] = simlarity(data[:,idx],passive_user[:,1:],passive_user[0,:,0],k),data[0,idx,0]
        users[idx,1:temp.shape[0]+1] = temp
        idx += 1
    return users

In [37]:
for i in range(len(list_file)):
    passive_user = cPickle.load( open( "passive_"+ str(list_file[i])+".pkl", "rb" ) )
    train,test = cPickle.load( open( "train_"+ str(i) +".pkl", "rb" ) ),cPickle.load( open( "test_"+ str(i)+".pkl", "rb" ) )
    users_matrix = k_neighbor(train,passive_user,40)
    cPickle.dump( users_matrix, open( "users_matrix_"+ str(i)+".pkl", "wb" ) )

# X_train,X_test | Y_train,Y_test

In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation

Using TensorFlow backend.


In [11]:
model = Sequential()
model.add(Dense(3, activation='relu', input_dim=5))
model.add(Dense(1,activation='linear'))
model.compile(optimizer='rmsprop',loss='mse')

In [12]:
model.save_weights('./mod/model.h5')

In [26]:
def train_error(model,train_x,train_y,test_x,test_y):
    lst = []
    list_err = []
    for i in range(len(train_x)):
        
        try:
            Tr_X = train_x[i]
            Tr_Y = np.reshape(train_y[i][:,-1],(train_y[i][:,-1].shape[0],1))
            Tt_X = test_x[i]
            Tt_Y = np.reshape(test_y[i][:,-1],(test_y[i][:,-1].shape[0],1))
            model.load_weights('./mod/model.h5')
#             model.fit(Tr_X,Tr_Y,epochs=500, batch_size=70, verbose= 0 )
            model.fit(Tr_X,Tr_Y,epochs=150, batch_size=train_x[i].shape[0], verbose= 0 )

            
            err,tp,fp,fn,tn = 0.0,0.0,0.0,0.0,0.0
        
            predicted = model.predict(Tt_X, batch_size=Tt_X.shape[0])
            for Pred,Actual in zip(predicted,Tt_Y):
                if Pred >= 7 and Actual >= 7:
                    tp+=1.0
                elif Pred < 7 and Actual >= 7:
                    fn+=1.0
                elif Pred >= 7 and Actual < 7:
                    fp+=1.0
                else:
                    tn+=1.0
        
            prec = tp/(tp+fp)
            recal = tp/(tp+fn)
            f_meas = (2*recal*prec)/(recal + prec)
            err = np.absolute((predicted - Tt_Y)).mean(axis=0)[0]
            list_err.append([err,prec,recal,f_meas])
        
        except:
            print("error : ",i)
                    
    list_err = array(list_err)
    
    return np.mean(list_err,axis=0)

In [27]:
def datashaping(users_matrix,train,passive_user,id_idx):
    X_train,Y_train = [],[]
    for row_idx in range(1,users_matrix[1:].shape[0]+1):
        x_train,y_train = [],[]
#         print("progress :: user completed :- ", row_idx)
        temp = train[0,row_idx,:].nonzero()[0]
        movie_pool = train[0,0,temp]
        for idx in range(1,movie_pool[1:].shape[0]+1):
            for usr in users_matrix[row_idx,1:]:
                if usr > 0 and passive_user[0,id_idx[usr],int(movie_pool[idx])] > 0 :
                    x_train.append(passive_user[:,id_idx[usr],int(movie_pool[idx])])
                    y_train.append(train[:,row_idx,int(temp[idx])])

        X_train.append(array(x_train))
        Y_train.append(array(y_train))
        
    return X_train,Y_train

In [28]:
lst_err = []
list_file = cPickle.load( open( "list_file.pkl", "rb" ) )
for i in range(len(list_file)):
    print("HI",i)
    passive_user = cPickle.load( open( "passive_"+ str(list_file[i])+".pkl", "rb" ) )
    train,test = cPickle.load( open( "train_"+ str(i) +".pkl", "rb" ) ),cPickle.load( open( "test_"+ str(i)+".pkl", "rb" ) )
    users_matrix = cPickle.load( open( "users_matrix_"+ str(i) +".pkl", "rb" ) )
    passive_user[:,1:,1:] = (passive_user[:,1:,1:])
    train[:,1:,1:] = (train[:,1:,1:])
    test[:,1:,1:] = (test[:,1:,1:])
    id_idx = {}
    for idx,ele in enumerate(passive_user[0,:,0]):
        id_idx[ele] = idx
    
    train_x,train_y = datashaping(users_matrix,train,passive_user,id_idx)
    test_x,test_y = datashaping(users_matrix,test,passive_user,id_idx)
    
    lst_err.append(train_error(model,train_x,train_y,test_x,test_y))

HI 0
error :  34
HI 1
HI 2
HI 3
HI 4
error :  6
HI 5
HI 6
HI 7
HI 8
error :  24
error :  32
HI 9


In [43]:
lst_err = cPickle.load( open( "lst_err.pkl", "rb" ) )

In [22]:
cPickle.dump( lst_err, open( "lst_err.pkl", "wb" ) )

In [8]:
list_file = cPickle.load( open( "list_file.pkl", "rb" ) )
passive_user = cPickle.load( open( "passive_"+ str(list_file[0])+".pkl", "rb" ) )
train,test = cPickle.load( open( "train_"+ str(0) +".pkl", "rb" ) ),cPickle.load( open( "test_"+ str(0)+".pkl", "rb" ) )
users_matrix = cPickle.load( open( "users_matrix_"+ str(0) +".pkl", "rb" ) )
passive_user[:,1:,1:] = (passive_user[:,1:,1:])
train[:,1:,1:] = (train[:,1:,1:])
test[:,1:,1:] = (test[:,1:,1:])
id_idx = {}
for idx,ele in enumerate(passive_user[0,:,0]):
    id_idx[ele] = idx
    
train_x,train_y = datashaping(users_matrix,train,passive_user,id_idx)
test_x,test_y = datashaping(users_matrix,test,passive_user,id_idx)
# train_error(model,train_x,train_y,test_x,test_y)

In [45]:
for i in range(9):
    lst_err[i][0] = lst_err[i][0] * 51
lst_err[-1][0] = lst_err[-1][0] * 25

In [46]:
lst_err

[array([ 2.35394782,  0.90462718,  0.89929684,  0.89444511]),
 array([ 2.22143838,  0.91288258,  0.88881221,  0.88991207]),
 array([ 2.23555299,  0.91531156,  0.88135118,  0.89129421]),
 array([ 2.3889509 ,  0.88119379,  0.8736491 ,  0.86369474]),
 array([ 2.31112709,  0.85447569,  0.90416508,  0.86253659]),
 array([ 2.26715263,  0.89485271,  0.88323387,  0.8828301 ]),
 array([ 2.51156562,  0.83280605,  0.9085413 ,  0.85866783]),
 array([ 2.21583983,  0.93055994,  0.904133  ,  0.9151052 ]),
 array([ 2.30062218,  0.92500828,  0.88813217,  0.90060183]),
 array([ 2.25068063,  0.94350535,  0.90116284,  0.91434626])]

In [38]:
lst_err

[array([ 0.04615584,  0.90462718,  0.89929684,  0.89444511]),
 array([ 0.04355762,  0.91288258,  0.88881221,  0.88991207]),
 array([ 0.04383437,  0.91531156,  0.88135118,  0.89129421]),
 array([ 0.04684217,  0.88119379,  0.8736491 ,  0.86369474]),
 array([ 0.04531622,  0.85447569,  0.90416508,  0.86253659]),
 array([ 0.04445397,  0.89485271,  0.88323387,  0.8828301 ]),
 array([ 0.04924638,  0.83280605,  0.9085413 ,  0.85866783]),
 array([ 0.04344784,  0.93055994,  0.904133  ,  0.9151052 ]),
 array([ 0.04511024,  0.92500828,  0.88813217,  0.90060183]),
 array([ 0.09002723,  0.94350535,  0.90116284,  0.91434626])]

In [48]:
np.savetxt("neural_network_10.csv", lst_err, delimiter=",")

# regression

In [4]:
model = Sequential()
model.add(Dense(1,activation='linear',input_dim=5))
model.compile(optimizer='rmsprop',loss='mse')

In [5]:
model.save_weights('./mod/model.h5')

In [15]:
train_x[1].shape[0]

81